In [23]:
from sklearn import *
import lightgbm as lgbm
import xgboost as xgb
import catboost as cat
from sklearn.model_selection import train_test_split
from sqlalchemy import column
import pandas as pd
import numpy as np
#!pip install optuna 
import optuna

from sklearn.compose import make_column_transformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import BayesianRidge

from lightgbm import LGBMRegressor
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [24]:
def convert_to_category(X):
    for col in X.columns:
        if ( (X[col].dtype != 'int64') and (X[col].dtype != 'float64') and (X[col].dtype != 'bool')):
            X[col] = X[col].astype('category')
            
    return X

def remove_outliers(df, column_names, n):
    for column_name in column_names:
        mean = df[column_name].mean()
        std = df[column_name].std()
        df = df[(df[column_name] > mean - n*std) & (df[column_name] < mean + n*std)]
    return df
    
def log_func(y, shift_var):
    return np.log(y+shift_var)

def exp_func(y, shift_var):
    return np.exp(y)-shift_var

In [89]:
import numpy as np 

def rmsle(y_true, y_pred):
    """
    Computes the Root Mean Squared Logarithmic Error 
    
    Args:
        y_true (np.array): n-dimensional vector of ground-truth values 
        y_pred (np.array): n-dimensional vecotr of predicted values 
    
    Returns:
        A scalar float with the rmsle value 
    
    Note: You can alternatively use sklearn and just do: 
        `sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5`
    """
    assert (y_true >= 0).all(), 'Received negative y_true values'
    assert (y_pred >= 0).all(), 'Received negative y_pred values'
    assert y_true.shape == y_pred.shape, 'y_true and y_pred have different shapes'
    y_true_log1p = np.log1p(y_true)  # log(1 + y_true)
    y_pred_log1p = np.log1p(y_pred)  # log(1 + y_pred)
    return np.sqrt(np.mean(np.square(y_pred_log1p - y_true_log1p)))

In [104]:
stores_train = pd.read_csv('data/stores_train.csv')
stores_test = pd.read_csv('data/stores_test.csv')

stores_train_num_busstops = pd.read_csv('data/other_data/stores_train_num_closest_busstop.csv')
stores_test_num_busstops = pd.read_csv('data/other_data/stores_test_num_closest_busstop.csv')

stores_train = pd.merge(stores_train, stores_train_num_busstops, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_busstops, on='store_id', how='left')

stores_train_closest_busstop = pd.read_csv('data/other_data/stores_train_closest_busstop.csv')
stores_test_closest_busstop = pd.read_csv('data/other_data/stores_test_closest_busstop.csv')

stores_train = pd.merge(stores_train, stores_train_closest_busstop, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_closest_busstop, on='store_id', how='left')

stores_train_closest_store_lv1 = pd.read_csv('data/other_data/stores_train_closest_store_lv1.csv')
stores_test_closest_store_lv1 = pd.read_csv('data/other_data/stores_test_closest_store_lv1.csv')

stores_train = pd.merge(stores_train, stores_train_closest_store_lv1, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_closest_store_lv1, on='store_id', how='left')

stores_train_closest_store_lv2 = pd.read_csv('data/other_data/stores_train_closest_store_lv2.csv')
stores_test_closest_store_lv2 = pd.read_csv('data/other_data/stores_test_closest_store_lv2.csv')

stores_train = pd.merge(stores_train, stores_train_closest_store_lv2, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_closest_store_lv2, on='store_id', how='left')

stores_train_closest_store_lv3 = pd.read_csv('data/other_data/stores_train_closest_store_lv3.csv')
stores_test_closest_store_lv3 = pd.read_csv('data/other_data/stores_test_closest_store_lv3.csv')

stores_train = pd.merge(stores_train, stores_train_closest_store_lv3, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_closest_store_lv3, on='store_id', how='left')

stores_train_closest_store_lv4 = pd.read_csv('data/other_data/stores_train_closest_store_lv4.csv')
stores_test_closest_store_lv4 = pd.read_csv('data/other_data/stores_test_closest_store_lv4.csv')

stores_train = pd.merge(stores_train, stores_train_closest_store_lv4, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_closest_store_lv4, on='store_id', how='left')

stores_train_num_closest_stores_lv1 = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv1.csv')
stores_test_num_closest_stores_lv1 = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv1.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv1, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv1, on='store_id', how='left')

stores_train_num_closest_stores_lv2 = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv2.csv')
stores_test_num_closest_stores_lv2 = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv2.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv2, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv2, on='store_id', how='left')

stores_train_num_closest_stores_lv3 = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv3.csv')
stores_test_num_closest_stores_lv3 = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv3.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv3, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv3, on='store_id', how='left')

stores_train_num_closest_stores_lv4 = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv4.csv')
stores_test_num_closest_stores_lv4 = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv4.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv4, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv4, on='store_id', how='left')

stores_train_copy= stores_train.copy()

plaace_hierarchy_copy = plaace_hierarchy.copy()
plaace_hierarchy_copy.drop(columns='sales_channel_name', inplace=True)
plaace_hierarchy_copy['lv1'] = plaace_hierarchy_copy['lv1'].astype('category')
plaace_hierarchy_copy['lv2'] = plaace_hierarchy_copy['lv2'].astype('category')


grunnkrets_copy = grunnkrets.copy()
grunnkrets_copy.rename(columns={'year': 'year_1'}, inplace=True)

grunnkrets_ages_copy = grunnkrets_ages.copy()
grunnkrets_ages_copy.rename(columns={'year': 'year_2'}, inplace=True)

grunnkrets_household_types_copy = grunnkrets_household_types.copy()
grunnkrets_household_types_copy.rename(columns={'year': 'year_3'}, inplace=True)
grunnkrets_household_types_copy[grunnkrets_household_types_copy.columns[2:]] = grunnkrets_household_types_copy[grunnkrets_household_types_copy.columns[2:]].astype('int64')

grunnkrets_household_income_copy = grunnkrets_household_income.copy()
grunnkrets_household_income_copy.rename(columns={'year': 'year_4', 'singles': 'singles_income','couple_without_children':'couple_without_children_income'}, inplace=True)
grunnkrets_household_income_copy.rename(columns={'singles': 'singles_income', }, inplace=True)

# set the values that are 0 to the lowest value in the column
# for column in grunnkrets_household_income_copy.columns[2:]:
#     grunnkrets_household_income_copy[column] = grunnkrets_household_income_copy[column].apply(lambda x: grunnkrets_household_income_copy[column].min() if x == 0 else x)
    


df = stores_train_copy

df = pd.merge(df, plaace_hierarchy_copy, on='plaace_hierarchy_id', how='left')
df = pd.merge(df, grunnkrets_copy, on='grunnkrets_id', how='left')
df = pd.merge(df, grunnkrets_ages_copy, on='grunnkrets_id', how='left')
df = pd.merge(df, grunnkrets_household_types_copy, on='grunnkrets_id', how='left')
df = pd.merge(df, grunnkrets_household_income_copy, on='grunnkrets_id', how='left')

df.drop_duplicates(subset=['store_id'], keep='first', inplace=True)

columns_to_drop = ['revenue',
                  'store_id',
                  'plaace_hierarchy_id',
                  'grunnkrets_id',
                  'geometry',
                  'year',
                  'year_1',
                  'year_2',
                  'year_3',
                  'year_4',
                  'address',
                  'store_name',
                  ]

X = df.drop(columns=columns_to_drop)
X = convert_to_category(X)


categorical_features = list(X.select_dtypes(include=['category']).columns)



# fill in missing values of float columns with mean

for col in X.columns:
    if ( (X[col].dtype == 'float64') or (X[col].dtype == 'int64')):
        X[col].fillna(X[col].mean(), inplace=True)
        
        
columns_to_scale_scaler = ['all_households', 'singles_income', 'couple_without_children_income', 'couple_with_children', 'other_households', 'single_parent_with_children',]
        
columns_to_scale_log = [
    'lat', 'lon', 
    'area_km2',
                     
        #'all_households', 'singles_income', 'couple_without_children_income', 'couple_with_children', 'other_households', 'single_parent_with_children',
                     
        'couple_children_0_to_5_years', 'couple_children_18_or_above', 'couple_children_6_to_17_years', 'couple_without_children', 'single_parent_children_0_to_5_years', 'single_parent_children_18_or_above', 'single_parent_children_6_to_17_years', 'singles',
                     
        'age_0', 'age_1', 'age_2', 'age_3', 'age_4', 'age_5', 'age_6', 'age_7', 'age_8', 'age_9', 'age_10', 'age_11',
        'age_12', 'age_13', 'age_14', 'age_15', 'age_16', 'age_17', 'age_18',
        'age_19', 'age_20', 'age_21', 'age_22', 'age_23', 'age_24', 'age_25',
        'age_26', 'age_27', 'age_28', 'age_29', 'age_30', 'age_31', 'age_32',
        'age_33', 'age_34', 'age_35', 'age_36', 'age_37', 'age_38', 'age_39',
        'age_40', 'age_41', 'age_42', 'age_43', 'age_44', 'age_45', 'age_46',
        'age_47', 'age_48', 'age_49', 'age_50', 'age_51', 'age_52', 'age_53',
        'age_54', 'age_55', 'age_56', 'age_57', 'age_58', 'age_59', 'age_60',
        'age_61', 'age_62', 'age_63', 'age_64', 'age_65', 'age_66', 'age_67',
        'age_68', 'age_69', 'age_70', 'age_71', 'age_72', 'age_73', 'age_74',
        'age_75', 'age_76', 'age_77', 'age_78', 'age_79', 'age_80', 'age_81',
        'age_82', 'age_83', 'age_84', 'age_85', 'age_86', 'age_87', 'age_88',
        'age_89', 'age_90',
                     
                     
        'num_closest_busstops',
        'closest_busstop', 'closest_store_lv1', 'closest_store_lv2',
        'closest_store_lv3', 'closest_store_lv4', 'num_closest_stores_lv1',
        'num_closest_stores_lv2', 'num_closest_stores_lv3',
        'num_closest_stores_lv4'
                    
                     
                     ]



scaler = preprocessing.StandardScaler()

#columns_to_scale_scaler = columns_to_scale_log

scaler.fit(X[columns_to_scale_scaler])

X[columns_to_scale_scaler] = scaler.transform(X[columns_to_scale_scaler])

for columns in columns_to_scale_log:
    X[columns] = np.log1p(X[columns])



full_pipeline = ColumnTransformer([('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)], remainder='passthrough')

encoder = full_pipeline.fit(X)
X_encoded = encoder.transform(X)

y = df.revenue
y = np.log1p(y)


### LGBM

In [109]:
def objective(trial,data=X,target=y):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.1, random_state=10)
    param = {
        'metric': 'rmse', 
        'random_state': 48,
        'n_estimators': trial.suggest_int('n_estimators', 100, 5000),
        #'n_estimators': 20000,
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.006,0.008,0.01,0.014,0.017,0.02]),
        'max_depth': trial.suggest_categorical('max_depth', [10,20,100]),
        'num_leaves' : trial.suggest_int('num_leaves', 1, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
    }
    model = lgbm.LGBMRegressor(**param)
    
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)
    
    preds = model.predict(test_x)
    
    test_y_exp = np.expm1(test_y)
    preds_exp = np.expm1(preds)
    
    error = rmsle( test_y_exp, preds_exp)
    
    return error

In [110]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2022-11-04 03:43:17,994] A new study created in memory with name: no-name-6e77584d-c7b0-49bd-8753-9a86811c42be
c:\Users\levit\.pyenv\pyenv-win\versions\3.10.7\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\levit\.pyenv\pyenv-win\versions\3.10.7\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\levit\.pyenv\pyenv-win\versions\3.10.7\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Re

Number of finished trials: 100
Best trial: {'n_estimators': 3168, 'reg_alpha': 0.0022182539366044883, 'reg_lambda': 0.01626240214227989, 'colsample_bytree': 0.6, 'subsample': 0.4, 'learning_rate': 0.006, 'max_depth': 100, 'num_leaves': 181, 'min_child_samples': 187}


In [111]:
params = study.best_trial.params

params

{'n_estimators': 3168,
 'reg_alpha': 0.0022182539366044883,
 'reg_lambda': 0.01626240214227989,
 'colsample_bytree': 0.6,
 'subsample': 0.4,
 'learning_rate': 0.006,
 'max_depth': 100,
 'num_leaves': 181,
 'min_child_samples': 187}

### XGBoost

In [112]:
def objective(trial,data=X_encoded,target=y):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.1,random_state=42)
    param = {
        'random_state': 48,
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 20000),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
        'eval_metric': 'rmse',

    }
    model = xgb.XGBRegressor(**param)
    
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)
    
    preds = model.predict(test_x)
    
    test_y_exp = np.expm1(test_y)
    preds_exp = np.expm1(preds)
    
    try:
        error = rmsle( test_y_exp, preds_exp)
    except:
        error = 77
    
    return error

In [113]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print('Best value:', study.best_value)

[I 2022-11-04 03:54:40,751] A new study created in memory with name: no-name-f4dd7409-46c0-469d-85b3-7f0c0684dbb9
c:\Users\levit\.pyenv\pyenv-win\versions\3.10.7\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
[I 2022-11-04 03:54:42,792] Trial 0 finished with value: 0.7583114954192478 and parameters: {'max_depth': 3, 'learning_rate': 0.02903716604381139, 'n_estimators': 2073, 'min_child_weight': 6, 'gamma': 1.5057142950003427e-08, 'subsample': 0.08849534794602339, 'colsample_bytree': 0.07175845021612295, 'reg_alpha': 0.0007283124614444005, 'reg_lambda': 5.056792143809415e-07}. Best is trial 0 with value: 0.7583114954192478.
c:\Users\levit\.pyenv\pyenv-win\versions\3.10.7\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibili

In [103]:
params = study.best_trial.params
best_result = study.best_value
print(params)
print(best_result)


{'max_depth': 5, 'learning_rate': 0.052335486885448576, 'n_estimators': 5398, 'min_child_weight': 1, 'gamma': 8.39873144547903e-07, 'subsample': 0.6721883930111987, 'colsample_bytree': 0.2371705470153603, 'reg_alpha': 0.25141043466302987, 'reg_lambda': 8.151934232465978e-07}
0.7264349071172609


# CatBoost

In [75]:
def objective(trial,data=X_encoded,target=y):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2,random_state=42)
 
    
    param = {}
    param['learning_rate'] = trial.suggest_discrete_uniform("learning_rate", 0.001, 0.02, 0.001)
    param['depth'] = trial.suggest_int('depth', 9, 15)
    param['l2_leaf_reg'] = trial.suggest_discrete_uniform('l2_leaf_reg', 1.0, 5.5, 0.5)
    param['min_child_samples'] = trial.suggest_categorical('min_child_samples', [1, 4, 8, 16, 32])
    param['grow_policy'] = 'Depthwise'
    param['iterations'] = 10000
    param['use_best_model'] = True
    param['eval_metric'] = 'RMSE'
    param['od_type'] = 'iter'
    param['od_wait'] = 20
    param['random_state'] = 42
    param['logging_level'] = 'Silent'
    model = cat.CatBoostRegressor(**param)
    
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)
    
    preds = model.predict(test_x)
    
    rmsle = mean_squared_error(test_y, preds,squared=False)
    
    return rmsle

In [76]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2022-11-04 02:03:28,578] A new study created in memory with name: no-name-f8d7db5c-1113-4c70-ab80-9961d4ecb5cf
[I 2022-11-04 02:08:46,569] Trial 0 finished with value: 0.7499954295113701 and parameters: {'learning_rate': 0.004, 'depth': 11, 'l2_leaf_reg': 3.0, 'min_child_samples': 16}. Best is trial 0 with value: 0.7499954295113701.
[I 2022-11-04 02:10:28,403] Trial 1 finished with value: 0.7508531133184271 and parameters: {'learning_rate': 0.011, 'depth': 9, 'l2_leaf_reg': 2.0, 'min_child_samples': 8}. Best is trial 0 with value: 0.7499954295113701.
[I 2022-11-04 02:13:06,997] Trial 2 finished with value: 0.7519462832710518 and parameters: {'learning_rate': 0.012, 'depth': 15, 'l2_leaf_reg': 2.5, 'min_child_samples': 32}. Best is trial 0 with value: 0.7499954295113701.
[I 2022-11-04 02:15:00,764] Trial 3 finished with value: 0.7537177408502436 and parameters: {'learning_rate': 0.018000000000000002, 'depth': 13, 'l2_leaf_reg': 2.5, 'min_child_samples': 8}. Best is trial 0 with value

KeyboardInterrupt: 

In [ ]:
params = study.best_trial.params

params